In [1]:
import pandas as pd
from zipfile import ZipFile

train_df = pd.read_csv("Train.csv")
test_df = pd.read_csv("Test.csv")

def addextension(nm):
    return nm+".jpg"

train_df.Image_ID = train_df.Image_ID.apply(addextension)
test_df.Image_ID = test_df.Image_ID.apply(addextension)

print(train_df.head())

          Image_ID            class   xmin   ymin  width  height
0  ID_007FAIEI.jpg  fruit_woodiness   87.0   87.5  228.0   311.0
1  ID_00G8K1V3.jpg  fruit_brownspot   97.5   17.5  245.0   354.5
2  ID_00WROUT9.jpg  fruit_brownspot  156.5  209.5  248.0   302.5
3  ID_00ZJEEK3.jpg    fruit_healthy  125.0  193.0  254.5   217.0
4  ID_018UIENR.jpg  fruit_brownspot   79.5  232.5  233.5   182.0


In [2]:
def unzip(nm):
    with ZipFile(nm,"r") as zip:
        zip.extractall()

unzip("Train_Images.zip")
unzip("Test_Images.zip")

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_gen = ImageDataGenerator(
    horizontal_flip = True,
    rotation_range = 10,
    zoom_range = 0.1,
    validation_split = 0.2,
)

train_ds = train_gen.flow_from_dataframe(
    directory = "Train_Images",
    dataframe = train_df,
    x_col = "Image_ID",
    y_col = "class",
    target_size = (256,256),
    batch_size = 32,
    class_mode = "categorical",
    shuffle = True,
    subset = "training",
)

val_ds = train_gen.flow_from_dataframe(
    directory = "Train_Images",
    dataframe = train_df,
    x_col = "Image_ID",
    y_col = "class",
    target_size = (256,256),
    batch_size = 32,
    class_mode = "categorical",
    shuffle = True,
    subset = "validation",
)

Found 3125 validated image filenames belonging to 3 classes.
Found 781 validated image filenames belonging to 3 classes.


In [4]:
from tensorflow.keras.applications import densenet, DenseNet201
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras import Model, Input
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.losses import CategoricalCrossentropy

rescale = densenet.preprocess_input
base_model = DenseNet201(input_shape=(256,256,3),include_top=False,weights="imagenet")
base_model.trainable = False

class Transfer_DenseNet201():
    def model(self,input):
        self.x = rescale(input)
        self.x = base_model(self.x,training=False)
        self.x = GlobalAveragePooling2D()(self.x)
        self.x = Dropout(0.2,seed=42)(self.x)
        self.output = Dense(3,activation="softmax")(self.x)
        self.model = Model(input,self.output,name="Transfer_DenseNet201")
        return self.model

TFDES201 = Transfer_DenseNet201()
model = TFDES201.model(Input(shape=(256,256,3)))
model.compile(RMSprop(),CategoricalCrossentropy(),metrics=["accuracy"])
model.summary()

Model: "Transfer_DenseNet201"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
tf.math.truediv (TFOpLambda) (None, 256, 256, 3)       0         
_________________________________________________________________
tf.nn.bias_add (TFOpLambda)  (None, 256, 256, 3)       0         
_________________________________________________________________
tf.math.truediv_1 (TFOpLambd (None, 256, 256, 3)       0         
_________________________________________________________________
densenet201 (Functional)     (None, 8, 8, 1920)        18321984  
_________________________________________________________________
global_average_pooling2d (Gl (None, 1920)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1920)     

In [5]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model


if __name__=="__main__":
    checkpoint = [
        ModelCheckpoint("makerere.hdf5",save_weights_only=False,save_best_only=True,monitor="val_accuracy")
    ]
    model.fit(train_ds,epochs=7,validation_data=val_ds,callbacks=checkpoint)
    best = load_model("makerere.hdf5")
    val_loss,val_acc = best.evaluate(val_ds)
    print("\nVal Accuracy: {:.2f} %".format(100*val_acc))
    print("Val Loss: {:.2f} %".format(100*val_loss))

Epoch 1/7
98/98 [==============================] - 73s 603ms/step - loss: 0.6109 - accuracy: 0.7600 - val_loss: 0.3734 - val_accuracy: 0.8732
Epoch 2/7
98/98 [==============================] - 50s 513ms/step - loss: 0.2736 - accuracy: 0.9168 - val_loss: 0.2060 - val_accuracy: 0.9347
Epoch 3/7
98/98 [==============================] - 53s 540ms/step - loss: 0.2038 - accuracy: 0.9357 - val_loss: 0.1601 - val_accuracy: 0.9501
Epoch 4/7
98/98 [==============================] - 51s 517ms/step - loss: 0.1637 - accuracy: 0.9504 - val_loss: 0.1428 - val_accuracy: 0.9539
Epoch 5/7
98/98 [==============================] - 50s 506ms/step - loss: 0.1445 - accuracy: 0.9536 - val_loss: 0.1227 - val_accuracy: 0.9616
Epoch 6/7
98/98 [==============================] - 51s 521ms/step - loss: 0.1335 - accuracy: 0.9533 - val_loss: 0.1175 - val_accuracy: 0.9680
Epoch 7/7
25/25 [==============================] - 12s 368ms/step - loss: 0.1155 - accuracy: 0.9629

Val Accuracy: 96.29 %
Val Loss: 11.55 %
